# Getting started

## Make sure Ollama is running

Go to [http://localhost:11434](http://localhost:11434) in your browser and make sure you get the response `Ollama is running`

# Accessing the Ollama API with requests

In [1]:
import requests

Let's start by getting the version of ollama we're running.

In [32]:
api_version = requests.get('http://localhost:11434/api/version')
api_version.json().get('version')

'0.5.13'

That's cool. Let's try chatting with it:

In [29]:
import json

data = {
    'model': 'tinyllama',
    'prompt': 'Why is the sky blue?'
}
response = requests.post('http://localhost:11434/api/generate', data=json.dumps(data))
if response.status_code == 200:
    for line in response.iter_lines():
        if line:
            response_data = json.loads(line)
            print(response_data.get("response", ""), end="", flush=True)

The sky is blue due to a mixture of sunlight and water vapor emitted by the earth's atmosphere. Sunlight is responsible for roughly 93% of the light that reaches Earth, while the remaining 7% is reflected from the surface of the planet. The remaining amount of light is absorbed by the atmospheric particles (including dust and water droplets) before it reaches the ground or sea. The balance between sunlight and atmospheric particles produces a colorful spectrum that we see as blue. However, there are other colors in the visible spectrum besides blue, such as green, yellow, and red.

🤔 As you can see, things got a little bit complicated. Because the reponse from ollama is actually a list of JSON objects, we need to parse through them to get the actual text response from each object. This is what the raw response looks like:

In [31]:
response.text

'{"model":"tinyllama","created_at":"2025-03-10T23:50:19.288912Z","response":"The","done":false}\n{"model":"tinyllama","created_at":"2025-03-10T23:50:19.310647Z","response":" sky","done":false}\n{"model":"tinyllama","created_at":"2025-03-10T23:50:19.331364Z","response":" is","done":false}\n{"model":"tinyllama","created_at":"2025-03-10T23:50:19.351242Z","response":" blue","done":false}\n{"model":"tinyllama","created_at":"2025-03-10T23:50:19.371313Z","response":" due","done":false}\n{"model":"tinyllama","created_at":"2025-03-10T23:50:19.391875Z","response":" to","done":false}\n{"model":"tinyllama","created_at":"2025-03-10T23:50:19.411081Z","response":" a","done":false}\n{"model":"tinyllama","created_at":"2025-03-10T23:50:19.432658Z","response":" mixture","done":false}\n{"model":"tinyllama","created_at":"2025-03-10T23:50:19.452221Z","response":" of","done":false}\n{"model":"tinyllama","created_at":"2025-03-10T23:50:19.472257Z","response":" sun","done":false}\n{"model":"tinyllama","created_

Besides the added complexity, we also had to wait awhile for the result to show up. You can show the result as it is received using streaming mode (`stream=True` in the `.post()` method). Try it out and notice how the words in the response appear sequentially instead of all at once.

In [33]:
data = {
    'model': 'tinyllama',
    'prompt': 'Why is the sky blue?'
}
with requests.post('http://localhost:11434/api/generate', data=json.dumps(data), stream=True) as response:
    if response.status_code == 200:
        for line in response.iter_lines():
            if line:
                response_data = json.loads(line)
                print(response_data.get("response", ""), end="", flush=True)

The sky blue color we see in the atmosphere of our planet Earth is caused by the presence of various trace elements and compounds that occur naturally in the earth's crust. These include iron, aluminum, magnesium, zinc, cadmium, sulfur, titanium, copper, molybdenum, and vanadium, among others. The colors seen in the sky depend on the chemical composition of the air we breathe, as well as the altitude at which we are standing. For instance, when the sun rises above a low altitude, the sunlight absorbs more light wavelengths from the atmosphere, leading to a brighter and bluer color of sky compared to higher altitudes where sunlight is reflected back into space. Ultimately, how much blue the sky appears depends on many factors such as wind patterns, temperature, humidity, and atmospheric conditions.

# Using the ollama Python library

Using the web API is perfectly acceptable, but it requires some extra work. Another option is to use the ollama Python library.

Let's start by importing the Ollama library. This library is modeled after the Ollama web API, making it easy to interact with Ollama through Python.

In [34]:
import ollama

## Pulling a model

You can use the `ollama.pull()` method to pull a new model. In this case, you should already have tinyllama downloaded, so this is just included for demonstration purposes.

In [5]:
# ollama.pull('tinyllama')

ProgressResponse(status='success', completed=None, total=None, digest=None)

## List available models

You can use the ollama library to list available models using the `ollama.list()` method.

In [6]:
ollama.list()

ListResponse(models=[Model(model='tinyllama:latest', modified_at=datetime.datetime(2025, 3, 7, 17, 50, 22, 405498, tzinfo=TzInfo(-06:00)), digest='2644915ede352ea7bdfaff0bfac0be74c719d5d5202acb63a6fb095b52f394a4', size=637700138, details=ModelDetails(parent_model='', format='gguf', family='llama', families=['llama'], parameter_size='1B', quantization_level='Q4_0')), Model(model='deepseek-r1:latest', modified_at=datetime.datetime(2025, 2, 7, 19, 53, 28, 828200, tzinfo=TzInfo(-06:00)), digest='0a8c266910232fd3291e71e5ba1e058cc5af9d411192cf88b6d30e92b6e73163', size=4683075271, details=ModelDetails(parent_model='', format='gguf', family='qwen2', families=['qwen2'], parameter_size='7.6B', quantization_level='Q4_K_M')), Model(model='llama3.2:latest', modified_at=datetime.datetime(2025, 2, 7, 19, 42, 22, 49375, tzinfo=TzInfo(-06:00)), digest='a80c4f17acd55265feec403c7aef86be0c25983ab279d83f3bcd3abbcb5b8b72', size=2019393189, details=ModelDetails(parent_model='', format='gguf', family='llama',

Well... 😷 that's kind of ugly. The API response is being cast as a couple of nested objects. Let's just get the names of the models from the API response.

response = ollama.list()
model_names = [response_model.model for response_model in response.models]
print(model_names)

## Start a chat

Chats can be performed either all at once or in "streaming" mode, just like before, but it's a little cleaner.

In [35]:
from ollama import chat

user_message =   {
    'role': 'user',
    'content': 'Why is the sky blue?',
}
response = chat(model='tinyllama', messages=[user_message])
print(response.message.content)

The sky blue color is caused by the presence of various compounds in the atmosphere, including blue light-absorbing substances like carbon nanotubes and titanium dioxide. These substances absorb blue light, which makes it appear blue in the sky. The blue hue that we observe in the sky is also influenced by the tilt of Earth's axis relative to the sun, as well as the angle at which the sun and its rays are received by the atmosphere. Ultimately, the exact reasons why the sky appears blue depend on various factors, such as atmospheric conditions, topography, and geological features.


## Streaming mode

In [38]:
user_message =   {
    'role': 'user',
    'content': 'Why is the sky blue?',
}
response = chat(model='tinyllama', messages=[user_message], stream=True)
for chunk in response:
    print(chunk.message.content, end='', flush=True)

The answer to the question "why is the sky blue?" is not definitive, as it varies depending on various factors, such as latitude, temperature, cloud formations, and atmospheric conditions. However, some common explanations include:

1. Sunlight - The sun's rays hitting the Earth's surface create the color of blue light (the visible spectrum).

2. Color in Nature - The sky is typically more blue than other colors because it reflects more blue light than red or green. This effect is known as "blue-green albedo."

3. Aircraft and Ships - Aircraft and ships flying overhead produce blue light from their engines, which can give the sky a blue appearance.

4. Geological Factors - The Earth's crust (the layer of rocky material beneath the surface) may absorb some sunlight, causing it to look darker in certain areas. However, this effect is generally small and difficult to measure accurately from space.

5. Chemical Reactions - Atmospheric gases like oxygen and nitrogen can absorb some blue lig